In [27]:
import time
import os
from torch.autograd import Variable
import torchvision
import torch
import random
import numpy as np
import numpy
import networks
# from my_args import  args
# from scipy.misc import imread, imsave
from imageio import imread
from PIL import Image

from AverageMeter import  *
import shutil
import matplotlib.pyplot as plt
import cv2
from tqdm import tqdm
from skimage.measure import compare_ssim
from scipy import ndimage

In [9]:
class Args():
    
    def __init__(self):
        self.filter_size = 4
        self.time_step = 0.5
        self.channels = 3
        self.netName = 'DAIN'
        self.use_cuda = True
        
        self.save_which = 1
        self.dtype = torch.cuda.FloatTensor
        
args = Args()

In [10]:
model = networks.__dict__[args.netName](    channel=args.channels,
                                    filter_size = args.filter_size ,
                                    timestep=args.time_step,
                                    training=False)

model = model.eval()

In [11]:
_ = model.eval()

In [12]:
if args.use_cuda:
    print('USING CUDA')
    model = model.cuda()

USING CUDA


In [13]:
args.SAVED_MODEL = '../../models/dain/best.pth'

In [14]:
##random_triplet():
import random
folder = random.choice(os.listdir('../../UCF-101/'))
files = os.listdir(os.path.join('../../UCF-101/', folder))

file = random.choice(files)
filepath = os.path.join('../../UCF-101/', folder, file)

filepath

'../../UCF-101/CliffDiving\\v_CliffDiving_g17_c04.avi'

In [15]:
def get_random_video():
    folder = random.choice(os.listdir('../../UCF-101/'))
    files = os.listdir(os.path.join('../../UCF-101/', folder))

    file = random.choice(files)
    filepath = os.path.join('../../UCF-101/', folder, file)
    return filepath

In [16]:
filepath = get_random_video()

def get_random_triplet(filepath):
    cap = cv2.VideoCapture(filepath)
    n_frames = cap.get(cv2.CAP_PROP_FRAME_COUNT)
    
    frame_start = np.random.randint(n_frames-2)
    
    frames = []
    for index in range(frame_start, frame_start+3):
        cap.set(cv2.CAP_PROP_POS_FRAMES, index)
        _, frame = cap.read()
        frames.append(frame)
        
    return frames
        
    

In [17]:
%%time
def triplet_generator(n=1000):
    for i in tqdm(range(n)):
        filepath = get_random_video()
        yield get_random_triplet(filepath)

Wall time: 0 ns


In [18]:
if os.path.exists(args.SAVED_MODEL):
    print("The testing model weight is: " + args.SAVED_MODEL)
    if not args.use_cuda:
        pretrained_dict = torch.load(args.SAVED_MODEL, map_location=lambda storage, loc: storage)
        # model.load_state_dict(torch.load(args.SAVED_MODEL, map_location=lambda storage, loc: storage))
    else:
        pretrained_dict = torch.load(args.SAVED_MODEL)
        # model.load_state_dict(torch.load(args.SAVED_MODEL))

    model_dict = model.state_dict()
    # 1. filter out unnecessary keys
    pretrained_dict = {k: v for k, v in pretrained_dict.items() if k in model_dict}
    # 2. overwrite entries in the existing state dict
    model_dict.update(pretrained_dict)
    # 3. load the new state dict
    model.load_state_dict(model_dict)
    # 4. release the pretrained dict for saving memory
    pretrained_dict = []
else:
    print("*****************************************************************")
    print("**** We don't load any trained weights **************************")
    print("*****************************************************************")

The testing model weight is: ../../models/dain/best.pth


In [19]:
use_cuda=args.use_cuda
save_which=args.save_which
dtype = args.dtype
unique_id =str(random.randint(0, 100000))

In [20]:
def get_padding(intWidth, intHeight, channel):
    
    if intWidth != ((intWidth >> 7) << 7):
        intWidth_pad = (((intWidth >> 7) + 1) << 7)  # more than necessary
        intPaddingLeft =int(( intWidth_pad - intWidth)/2)
        intPaddingRight = intWidth_pad - intWidth - intPaddingLeft
    else:
        intWidth_pad = intWidth
        intPaddingLeft = 32
        intPaddingRight= 32

    if intHeight != ((intHeight >> 7) << 7):
        intHeight_pad = (((intHeight >> 7) + 1) << 7)  # more than necessary
        intPaddingTop = int((intHeight_pad - intHeight) / 2)
        intPaddingBottom = intHeight_pad - intHeight - intPaddingTop
    else:
        intHeight_pad = intHeight
        intPaddingTop = 32
        intPaddingBottom = 32
    
    
    return intPaddingLeft, intPaddingRight , intPaddingTop, intPaddingBottom




def preprocess_image(im):
    X = torch.from_numpy( np.transpose(im, (2,0,1)).astype('float32')/255.0).type(dtype)
    
    intWidth = X.size(2)
    intHeight = X.size(1)
    channel = X.size(0)
    
    
    intPaddingLeft, intPaddingRight , intPaddingTop, intPaddingBottom = get_padding(intWidth, intHeight, channel)
    
    pader = torch.nn.ReplicationPad2d([intPaddingLeft, intPaddingRight , intPaddingTop, intPaddingBottom])
    
    torch.set_grad_enabled(False)
    X = Variable(torch.unsqueeze(X,0))
    X = pader(X)
    
    return X, (intPaddingLeft, intPaddingRight , intPaddingTop, intPaddingBottom)
    

    
    
def interpolate(im1, im2):
    X0, pad_0 = preprocess_image(im1)
    X1, pad_1 = preprocess_image(im2)
    
    y_ = torch.FloatTensor()

    assert pad_0 == pad_1
    
    intPaddingLeft, intPaddingRight , intPaddingTop, intPaddingBottom = pad_0
    
    if use_cuda:
        X0 = X0.cuda()
        X1 = X1.cuda()
        
    y_s,offset,filter = model(torch.stack((X0, X1),dim = 0))
    y_ = y_s[save_which]
    
    
    if use_cuda:
        X0 = X0.data.cpu().numpy()
        if not isinstance(y_, list):
            y_ = y_.data.cpu().numpy()
        else:
            y_ = [item.data.cpu().numpy() for item in y_]
        offset = [offset_i.data.cpu().numpy() for offset_i in offset]
        filter = [filter_i.data.cpu().numpy() for filter_i in filter]  if filter[0] is not None else None
        X1 = X1.data.cpu().numpy()
    else:
        X0 = X0.data.numpy()
        if not isinstance(y_, list):
            y_ = y_.data.numpy()
        else:
            y_ = [item.data.numpy() for item in y_]
        offset = [offset_i.data.numpy() for offset_i in offset]
        filter = [filter_i.data.numpy() for filter_i in filter]
        X1 = X1.data.numpy()
        
    X0 = np.transpose(255.0 * X0.clip(0,1.0)[0, :, intPaddingTop:intPaddingTop+intHeight, intPaddingLeft: intPaddingLeft+intWidth], (1, 2, 0))


    y_ = [np.transpose(255.0 * item.clip(0,1.0)[:, intPaddingTop:intPaddingTop+intHeight, intPaddingLeft: intPaddingLeft+intWidth], (1, 2, 0)) for item in y_]


    offset = [np.transpose(offset_i[0, :, intPaddingTop:intPaddingTop+intHeight, intPaddingLeft: intPaddingLeft+intWidth], (1, 2, 0)) for offset_i in offset]
    filter = [np.transpose(
        filter_i[0, :, intPaddingTop:intPaddingTop + intHeight, intPaddingLeft: intPaddingLeft + intWidth],
        (1, 2, 0)) for filter_i in filter]  if filter is not None else None
    X1 = np.transpose(255.0 * X1.clip(0,1.0)[0, :, intPaddingTop:intPaddingTop+intHeight, intPaddingLeft: intPaddingLeft+intWidth], (1, 2, 0))
    
    return y_[0]
    
    
def interpolate_old(im1, im2):
    X0 = torch.from_numpy( np.transpose(im1, (2,0,1)).astype('float32')/255.0).type(dtype)
    X1 = torch.from_numpy( np.transpose(im2, (2,0,1)).astype('float32')/255.0).type(dtype)

    y_ = torch.FloatTensor()

    assert (X0.size(1) == X1.size(1))
    assert (X0.size(2) == X1.size(2))
    
    intWidth = X0.size(2)
    intHeight = X0.size(1)
    channel = X0.size(0)

    if intWidth != ((intWidth >> 7) << 7):
        intWidth_pad = (((intWidth >> 7) + 1) << 7)  # more than necessary
        intPaddingLeft =int(( intWidth_pad - intWidth)/2)
        intPaddingRight = intWidth_pad - intWidth - intPaddingLeft
    else:
        intWidth_pad = intWidth
        intPaddingLeft = 32
        intPaddingRight= 32

    if intHeight != ((intHeight >> 7) << 7):
        intHeight_pad = (((intHeight >> 7) + 1) << 7)  # more than necessary
        intPaddingTop = int((intHeight_pad - intHeight) / 2)
        intPaddingBottom = intHeight_pad - intHeight - intPaddingTop
    else:
        intHeight_pad = intHeight
        intPaddingTop = 32
        intPaddingBottom = 32

    pader = torch.nn.ReplicationPad2d([intPaddingLeft, intPaddingRight , intPaddingTop, intPaddingBottom])
    
    torch.set_grad_enabled(False)
    X0 = Variable(torch.unsqueeze(X0,0))
    X1 = Variable(torch.unsqueeze(X1,0))
    X0 = pader(X0)
    X1 = pader(X1)
    
    
    if use_cuda:
        X0 = X0.cuda()
        X1 = X1.cuda()
    model.eval()
    y_s,offset,filter = model(torch.stack((X0, X1),dim = 0))
    y_ = y_s[save_which]
    
    
    if use_cuda:
        X0 = X0.data.cpu().numpy()
        if not isinstance(y_, list):
            y_ = y_.data.cpu().numpy()
        else:
            y_ = [item.data.cpu().numpy() for item in y_]
        offset = [offset_i.data.cpu().numpy() for offset_i in offset]
        filter = [filter_i.data.cpu().numpy() for filter_i in filter]  if filter[0] is not None else None
        X1 = X1.data.cpu().numpy()
    else:
        X0 = X0.data.numpy()
        if not isinstance(y_, list):
            y_ = y_.data.numpy()
        else:
            y_ = [item.data.numpy() for item in y_]
        offset = [offset_i.data.numpy() for offset_i in offset]
        filter = [filter_i.data.numpy() for filter_i in filter]
        X1 = X1.data.numpy()
        
    X0 = np.transpose(255.0 * X0.clip(0,1.0)[0, :, intPaddingTop:intPaddingTop+intHeight, intPaddingLeft: intPaddingLeft+intWidth], (1, 2, 0))


    y_ = [np.transpose(255.0 * item.clip(0,1.0)[:, intPaddingTop:intPaddingTop+intHeight, intPaddingLeft: intPaddingLeft+intWidth], (1, 2, 0)) for item in y_]
    # 0 weggehaald als index

    offset = [np.transpose(offset_i[0, :, intPaddingTop:intPaddingTop+intHeight, intPaddingLeft: intPaddingLeft+intWidth], (1, 2, 0)) for offset_i in offset]
    filter = [np.transpose(
        filter_i[0, :, intPaddingTop:intPaddingTop + intHeight, intPaddingLeft: intPaddingLeft + intWidth],
        (1, 2, 0)) for filter_i in filter]  if filter is not None else None
    X1 = np.transpose(255.0 * X1.clip(0,1.0)[0, :, intPaddingTop:intPaddingTop+intHeight, intPaddingLeft: intPaddingLeft+intWidth], (1, 2, 0))
    del X0; del X1
    return y_[0]


In [36]:
sx = ndimage.sobel(imagei, axis=0, mode='constant')
sy = ndimage.sobel(imagei, axis=1, mode='constant')
Nabla_x = np.stack([sx,sy])
# imagei
ssq = np.power(Nabla_x, 2).sum()

def im_grad(image):
    sx = ndimage.sobel(image, axis=0, mode='constant')
    sy = ndimage.sobel(image, axis=1, mode='constant')
    nabla = np.stack([sx,sy])
    return nabla

In [35]:
ssq

25552333000.0

In [32]:
Nabla_x.shape

(2, 240, 320, 3)

In [43]:
def psnr(y_hat, y):
    mse = np.mean((y_hat-y)**2)
    R = np.max([y_hat, y])
    return 10 * np.log10(R**2 / mse)


def ssim(y_hat, y):
    None
    
def ie(y_hat, y):
    return np.sqrt(np.mean((y_hat-y)**2))

def nie(y_hat, y, eps=1.0):
    diff = np.power(y_hat-y, 2)
    norm = (im_grad(y)**2).sum()
    
    return np.sqrt(np.mean(diff/(norm+eps)))
    

In [44]:
ie(interpolated, imagei)

12.908239

In [23]:
gen = triplet_generator(n=100)

In [24]:
model = model.eval()
psnrs = []
ssims = []
for image1, imagei, image2 in gen:
    
    imagei = imagei.astype('float32')

    interpolated= interpolate_old(image1, image2)
    
    psnr_score = psnr(interpolated, imagei)
    
    ssim_score = compare_ssim(interpolated, imagei, multichannel=True)
    psnrs.append(psnr_score)
    ssims.append(ssim_score)
#     print(psnr_score, ssim_score)

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [00:37<00:00,  2.76it/s]


In [18]:
np.mean(psnrs), np.mean(ssims)

(33.37709865794216, 0.8417291325258767)

In [19]:
np.mean(psnrs), np.mean(ssims)

(32.18444912304881, 0.8188654567170967)